In [56]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import json
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pickle

In [31]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

## Test Case with 1 Town
* locate all necessary items on webpage

In [20]:
page = requests.get("https://www.alltrails.com/explore/us/california/los-gatos").text
soup = BeautifulSoup(page, 'lxml')

In [24]:
soup.find('div', id='item-info')

<!DOCTYPE html>
<html class="explore index" dir="ltr" lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# alltrails: http://ogp.me/ns/fb/alltrails#">
<script data-apikey="7ee34601ee591f69f6540476945373cf" src="//d2wy8f7a9ursnm.cloudfront.net/bugsnag-2.min.js"></script>
<title class="xlate-none">Map of Trails near Los Gatos, California | AllTrails</title>
<link href="https://www.alltrails.com/explore/us/california/los-gatos" hreflang="en" rel="alternate"/>
<link href="https://www.alltrails.com/fr/explore/us/california/los-gatos" hreflang="fr" rel="alternate"/>
<link href="https://www.alltrails.com/de/explore/us/california/los-gatos" hreflang="de" rel="alternate"/>
<link href="https://www.alltrails.com/es/explore/us/california/los-gatos" hreflang="es" rel="alternate"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Con

In [78]:
#find review text
review_list = []
for each in soup.findAll(itemprop="reviewBody"):
    review = each.text
    review_list.append(review)
    
print(review_list)

['', 'Good hill run. Go clockwise as the uphills are more gradual that way. First mile is just a slight incline following the creek and is nicely shaded. At around mile one the gradient picks up and it’s a good workout, still  in shade. You top out at mile 2.5 and run along the summit on a fire road. There are some steep downs and nice views. You drop down to the road about 1/3 of a mile from where you parked, so it’s a leisurely trot back to the car. It’s good for dogs too,  it watch for mountain bikers if you have your dog off leash. It gets pretty hot on summer days but the first half is under shade.', '', 'Definitely take the trail clockwise!  Middle 1/3 of trail is exposed fire road - boring!  Late 1/3 of trail dotted with views of Lexington reservoir! Not crowded at all. Went at 1:30 on a Sunday! Good quick cardio and nice close trail to Silly Valley. Enjoy!', 'Parking is alongside the road; when I arrived around 10:45 am on a Sunday, there were a few spots open. This trail is do

In [ ]:
soup.select('span.distance-icon')[0].text

In [ ]:
# parsing tag cloud - hike attributes
empty_list = []
for each in soup.find('section', class_='tag-cloud'):
    print(each)

In [ ]:
# first draft - hike attributes
trail_perks = []
    trail_stats = {'distance':0,'perks':trail_perks,'reviews':review_list}

In [183]:
# first loop for user reviews from Los Gatos, CA only
for trail in trail_list:
    url = "https://www.alltrails.com"+trail
    trail_soup = scrape_trails(url)
    trail_stats, trail_name, user_dict = get_attributes(trail_soup)
    print(trail_stats)     
    driver.close()

{'hike_name': 'Lexington Reservoir Trail', 'hike_difficulty': 'MODERATE', 'stars': '4.2', 'num_reviews': '323', 'hike_region': 'Sierra Azul Open Space Preserve', 'total_distance': '            5.0 miles', 'elevation_gain': '            1,204 feet', 'route_type': 'Loop', 'hike_attributes': ['dogs on leash', 'kid friendly', 'hiking', 'mountain biking', 'nature trips', 'paddle sports', 'walking', 'bird watching', 'running', 'views', 'wild flowers'], 'ratings': [{'Tommy Geraghty': '4'}, {'Timotjy Morey': '4'}, {'Swami Ramachandran': '4'}, {'Tina Engel': '4'}, {'L P': '3'}, {'Brynnan Green': '3'}, {'Jorge Zubiran': '4'}, {'guy m': '5'}, {'Lisa Frank': '3'}, {'Burtman Burt': '4'}, {'Erin Rand': '4'}, {'Fernando Cruz': '5'}, {'Patrice Lucas': '4'}, {'Spencer Todd': '4'}, {'Daniela Landey': '4'}, {'Kelley Gallaher': '4'}, {'Dadise Aleman': '5'}, {'Ravi Sunkara': '4'}, {'Joseph Bernard': '4'}, {'John Huang': '4'}, {'Davis Meiley': '5'}, {'Sina Mohseni': '3'}, {'Tara McHugh': '4'}, {'Bettina Ale

InvalidSessionIdException: Message: invalid session id


# Scrape for All of California
* scrape search results for all 'California' trails
* scrape each California trail
    * trail attributes
    * trail reviews

In [52]:
# function for selenium to load all hike links from California
def get_all_hikes(url):
    '''input: URL
       output: fully loaded webpage with all hike links for URL'''
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    # Selects Decline in pop-up
    driver.find_element_by_xpath("//a[contains(text(),'Decline')]").click()
    count = 0
    while count < 60:
        try:
            load_more_hikes = WebDriverWait(driver, 8).until(EC.visibility_of_element_located((By.XPATH,"//div[@id='load_more'] [@class='feed-item load-more trail-load'][//a]")))
            # clicks to load more
            load_more_hikes.click()
            time.sleep(2)
            count += 1
        except:
            break

    soup = BeautifulSoup(driver.page_source, 'lxml')
    return soup

In [53]:
# get list of all trail URLS from CA page
trail_urls = []
hike_soup = get_all_hikes('https://www.alltrails.com/us/california')

for trail in hike_soup.find_all(class_='item-link', itemprop='url'):
    url = trail.get('href')
    trail_urls.append(url)

In [54]:
# total number of scraped URLs
len(trail_urls)

1464

In [57]:
# with open('trail_urls.pickle', 'wb') as to_write:
#      pickle.dump(trail_urls, to_write)

In [157]:
# function for selenium to load all trail reviews
def scrape_trails(url):
    '''input: hike URL
       output: webpage loaded with all user reviews for trail'''
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    finished = False
    count = 0
    while not finished:
        try:
            load_ratings = WebDriverWait(driver, 6).until(EC.visibility_of_element_located((By.XPATH,"//div[@id='load_more'] [@class='feed-item load-more'][//a]")))
            load_ratings.click()
            time.sleep(6)
            newcount = len(driver.find_elements_by_class_name('feed-user-content.rounded'))
            if newcount > count:
                count = newcount
            else:
                finished = True
                
        except:
            break  
            
    soup = BeautifulSoup(driver.page_source, "lxml")
    
    return soup

In [139]:
# function to scrape each trail page for attributes
def get_attributes(trail_soup):
    '''input: BeautifulSoup webpage
    output: Returns 3 items - all trail attributes, name of trail, all user reviews'''
    header = trail_soup.find('div', id='title-and-menu-box')
    trail_name = header.findChild('h1').text
    difficulty = header.findChild('span').text
    stars = header.findChild('meta')['content']
    num_reviews = header.find('span', itemprop='reviewCount').text
    area = trail_soup.select('div.trail-rank')
    try:
        hike_region = area[0].findChild('span', itemprop='name').text
    except:
        hike_region = area[0].findChild('a').text
    try:
        distance = trail_soup.select('span.distance-icon')[0].text
        distance = re.search('\\n\\n(.+?)\\n', distance).group(1)
    except:
        distance = None
    try:
        elevation_gain = trail_soup.select('span.elevation-icon')[0].text
        elevation_gain = re.search('\\n\\n(.+?)\\n', elevation_gain).group(1)
    except:
        elevation_gain = None
    try:
        route_type = trail_soup.select('span.route-icon')[0].text
        route_type = re.search(':\\n(.+?)\\n', route_type).group(1)
    except:
        route_type = None
    tags = trail_soup.select('section.tag-cloud')[0].findChildren('h3')
    hike_attributes = []
    for tag in tags:
        hike_attributes.append(tag.text)
        
    user_dict = {}
    users = trail_soup.select('div.feed-user-content.rounded')
    for user in users:
        if user.find('span', itemprop='author') != None:
            user_name = user.find('span', itemprop='author').text
            user_name = user_name.replace('.', '')
            user_dict[user_name] = {'rating':None,'review':None}
            try:
                rating = user.find('span', itemprop="reviewRating").findChildren('meta')[0]['content']
                user_dict[user_name]['rating'] = rating
#                 user_ratings.append({user_name: rating})
            except:
                pass
        if user.find('p',itemprop='reviewBody') != None:
            try:
                review = user.find('p',itemprop='reviewBody').text
                if review != '':
                    user_dict[user_name]['review'] = review
                else:
                    pass
#                     user_dict[user_name]['review'] = np.nan
#                 user_reviews.append({user_name: review})
            except:
                pass
            
    trail_stats = {}
    # create dictionary with all of attributes to turn into dataframe
    trail_stats[trail_name] = {'hike_difficulty':difficulty,'stars':stars,'num_reviews':num_reviews,'hike_region': hike_region,
                               'total_distance':distance,'elevation_gain':elevation_gain,'route_type':route_type,'hike_attributes':hike_attributes}
    
    #merge trail dataframe with user dataframe

    return trail_stats, trail_name, user_dict

In [63]:
list(set(trail_urls))

['/trail/us/california/tin-mine-canyon-trail?ref=result-card',
 '/trail/us/california/van-damme-state-park-fern-canyon-trail?ref=result-card',
 '/trail/us/california/folsom-trail?ref=result-card',
 '/trail/us/california/santa-paula-canyon-and-punch-bowls-trail?ref=result-card',
 '/trail/us/california/schabarum?ref=result-card',
 '/trail/us/california/pioneer-trail--2?ref=result-card',
 '/trail/us/california/phantom-falls?ref=result-card',
 '/trail/us/california/crockett-hills-flow-trail?ref=result-card',
 '/trail/us/california/eagle-view-trail--2?ref=result-card',
 '/trail/us/california/rocky-peak-rocky-peak-chumash-trails?ref=result-card',
 '/trail/us/california/kings-creek-falls-trail?ref=result-card',
 '/trail/us/california/tahquitz-peak-via-devils-slide-and-caramba-trail?ref=result-card',
 '/trail/us/california/columbia-rock-trail?ref=result-card',
 '/trail/us/california/fort-point-baker-beach-coast-trail?ref=result-card',
 '/trail/us/california/sentinel-dome-and-taft-point-loop?re

In [173]:
# saving unique trail URLs
ca_trails = list(set(trail_urls))

In [146]:
# create empty dataframes for scraped data to fill
user_agg_df = pd.DataFrame()
trail_agg_df = pd.DataFrame()

In [ ]:
# loop for user reviews
# VERY time consuming for large number of trails
# stopped webscrape after 1050 trails due to time constraints
for trail in ca_trails[1011:]:
    url = "https://www.alltrails.com"+trail
    trail_soup = scrape_trails(url)
    try:
        trail_stats, trail_name, user_dict = get_attributes(trail_soup)
        # convert user dict to dataframe
        user_df = pd.DataFrame(user_dict)
        user_df = user_df.transpose()
        user_df = user_df.reset_index().rename(columns={'index':'username'})
        user_df['trail_name'] = trail_name
        user_agg_df = user_agg_df.append(user_df, ignore_index=True)
        # convert trail stats to dataframe and append
        trail_df = pd.DataFrame(trail_stats)
        trail_df = trail_df.transpose()
        trail_df = trail_df.reset_index().rename(columns={'index':'trail_name'})
        trail_agg_df = trail_agg_df.append(trail_df, ignore_index=True)
    except:
        pass

In [205]:
# look at all non-null reviews
user_agg_df[user_agg_df['review'].notnull()]

,username,rating,review,trail_name
1,Gabe Flores,5,"This is a very pleasant little hike, more beau...",Tin Mine Canyon Trail
2,Steve Blakeney,4,Fun and scenic trail with little rock pagodas ...,Tin Mine Canyon Trail
3,Nathan Koeneman,3,didn't make it all the way as the rain washed ...,Tin Mine Canyon Trail
4,Sarah Huntington,5,This trail was absolutely beautiful. The trail...,Tin Mine Canyon Trail
5,Louis Davis,5,Good hike today. Many parts of the trail have ...,Tin Mine Canyon Trail
...,...,...,...,...
236087,Joshua Elkin,5,Awesome hike. Did the same loop as Dan S (see ...,Mineral King to Little Five Lakes Trail
236088,Mike Row,4,Went in late Sept. Was cold and got snowed on...,Mineral King to Little Five Lakes Trail
236089,Dan S,5,A couple things to note:\r\n$20 per car to dri...,Mineral King to Little Five Lakes Trail
236090,Macgregor Pearce,5,Just got back from this loop. Absolutely beaut...,Mineral King to Little Five Lakes Trail


In [298]:
# filter dataframe for reviews longer than 20 characters
mask = (review_df['review_length'] > 20)
ok_review_df = review_df.loc[mask]

In [299]:
# look at only reviews that have a length of more than 20 characters
ok_review_df.sort_values(by=['review_length'])

,username,rating,review,trail_name,review_length
10895,BreMo BreMo Kassy&Lo,5,"easy, lots of stairs",Cascade Falls Trail,21.0
46,Joseph Cesneros,5,Beautiful like always,Santa Margarita River Trail,21.0
28949,K T,5,great hiking for dogs,McLeod Lake Trail,21.0
8065,Kahn Ryburn-Distel,3,"Beautiful, easy hike!",Rainbow Falls via Devils Postpile Trail,21.0
19840,Bret Van Blaricom,4,"easy, convenient, fun",Hidden Falls Trail,21.0
...,...,...,...,...,...
9885,Luke's HomeTube,5,"A friend and I started this hike on Wednesday,...",Mount Whitney Trail,4849.0
9812,Josh Hunsaker,5,Wednesday 10/18\r\nI drove up from LA and arri...,Mount Whitney Trail,4903.0
4873,Satya Upad,5,"This is a beautiful trail, and probably that's...",Lost Coast Trail: Mattole to Black Sands Beach,4990.0
9610,Rachel Crompton,5,We did this as a day hike in a team of 4. I've...,Mount Whitney Trail,5000.0


In [210]:
# with open('trail_agg_df.pickle', 'wb') as to_write:
#       pickle.dump(trail_agg_df, to_write)

In [208]:
trail_agg_df
# left off on 1025

,trail_name,elevation_gain,hike_attributes,hike_difficulty,hike_region,num_reviews,route_type,stars,total_distance
0,Tin Mine Canyon Trail,715 feet,"[dogs on leash, hiking, walking, partially pav...",MODERATE,Cleveland National Forest,276,Out & Back,4.2,4.5 miles
1,Fern Canyon Scenic Trail,826 feet,"[camping, hiking, nature trips, walking, bird ...",MODERATE,Van Damme Beach State Park,144,Loop,4.5,8.3 miles
2,Folsom Trail,265 feet,"[dogs on leash, kid friendly, hiking, mountain...",MODERATE,Folsom Lake State Recreation Area,121,Out & Back,4.1,9.2 miles
3,Santa Paula Canyon and Punch Bowls Trail,787 feet,"[dogs on leash, camping, hiking, horseback rid...",MODERATE,Los Padres National Forest,631,Out & Back,4.4,7.0 miles
4,Schabarum Trail,305 m,"[dogs on leash, hiking, mountain biking, natur...",MODERATE,Walnut Ranch Park,196,Loop,4.3,5.8 km
...,...,...,...,...,...,...,...,...,...
1020,Griffith Park Old Zoo Loop,419 feet,"[hiking, nature trips, walking, bird watching,...",easy,Griffith Park,193,Loop,4.2,2.7 miles
1021,Brown Mountain Dam Waterfall,656 feet,"[dogs on leash, backpacking, hiking, mountain ...",moderate,Angeles National Forest,123,Out & Back,4.5,8.4 miles
1022,San Diego Sea to Sea Trail via Camino Ruiz Park,403 feet,"[dogs on leash, hiking, mountain biking, bird ...",easy,Los Penasquitos Canyon Preserve,142,Out & Back,4.2,4.8 miles
1023,Partington Cove Trail,347 feet,"[hiking, nature trips, beach, river, views, wa...",moderate,"Big Sur, California",205,Out & Back,4.6,1.1 miles


In [209]:
user_agg_df

,username,rating,review,trail_name
0,Stephanie Toole,4,None,Tin Mine Canyon Trail
1,Gabe Flores,5,"This is a very pleasant little hike, more beau...",Tin Mine Canyon Trail
2,Steve Blakeney,4,Fun and scenic trail with little rock pagodas ...,Tin Mine Canyon Trail
3,Nathan Koeneman,3,didn't make it all the way as the rain washed ...,Tin Mine Canyon Trail
4,Sarah Huntington,5,This trail was absolutely beautiful. The trail...,Tin Mine Canyon Trail
...,...,...,...,...
241223,Matthew Castelli,4,None,Sanborn Skyline Park to Castle Rock State Park
241224,Kate Momotova,4,None,Sanborn Skyline Park to Castle Rock State Park
241225,ChasingAdventure29 com,5,None,Sanborn Skyline Park to Castle Rock State Park
241226,Anita Perez,5,None,Sanborn Skyline Park to Castle Rock State Park


In [151]:
user_agg_df['trail_name'].nunique()

122

In [ ]:
# with open('user_agg_df', 'wb') as to_write:
#      pickle.dump(user_agg_df, to_write)

In [211]:
# list of trails to debug - function did not scrape
#https://www.alltrails.com/trail/us/california/homestead-to-blue-ridge-loop--2?ref=result-card
#https://www.alltrails.com/trail/us/california/cleos-bath-trail-via-pinecrest-lake-loop?ref=result-card
#https://www.alltrails.com/trail/us/california/woods-winnemucca-and-round-top-lakes?ref=result-card
# https://www.alltrails.com/trail/us/california/alamere-falls-via-coast-trail-from-palomarin-trailhead?ref=result-card
# https://www.alltrails.com/trail/us/california/temescal-canyon-will-rogers-loop-trail?ref=result-card
# volcan mountain trail
#https://www.alltrails.com/trail/us/california/burney-falls-loop-trail?ref=result-card
# point lobos trail
#https://www.alltrails.com/trail/us/california/shadow-and-ediza-lakes-trail?ref=result-card
#https://www.alltrails.com/trail/us/california/paradise-lake-trail?ref=result-card
# '/trail/us/california/paradise-lake-trail?ref=result-card',

In [156]:
list(set(trail_urls))

['/trail/us/california/tin-mine-canyon-trail?ref=result-card',
 '/trail/us/california/van-damme-state-park-fern-canyon-trail?ref=result-card',
 '/trail/us/california/folsom-trail?ref=result-card',
 '/trail/us/california/santa-paula-canyon-and-punch-bowls-trail?ref=result-card',
 '/trail/us/california/schabarum?ref=result-card',
 '/trail/us/california/pioneer-trail--2?ref=result-card',
 '/trail/us/california/phantom-falls?ref=result-card',
 '/trail/us/california/crockett-hills-flow-trail?ref=result-card',
 '/trail/us/california/eagle-view-trail--2?ref=result-card',
 '/trail/us/california/rocky-peak-rocky-peak-chumash-trails?ref=result-card',
 '/trail/us/california/kings-creek-falls-trail?ref=result-card',
 '/trail/us/california/tahquitz-peak-via-devils-slide-and-caramba-trail?ref=result-card',
 '/trail/us/california/columbia-rock-trail?ref=result-card',
 '/trail/us/california/fort-point-baker-beach-coast-trail?ref=result-card',
 '/trail/us/california/sentinel-dome-and-taft-point-loop?re

## MongoDB

In [ ]:
import quandl
import pymongo
from pymongo import MongoClient

In [ ]:
# trail_agg_df = pd.DataFrame()

#     agg_df = pd.DataFrame(user_dict)
#     agg_df = agg_df.transpose()
#     agg_df = agg_df.reset_index().rename(columns={'index':'username'})
#     agg_df['trail_name'] = trail_name
#     trail_agg_df = trail_agg_df.append(agg_df, ignore_index=True)